In this note, I test my codes of calculating the vector spherical harmonics.

In [1]:
from scipy.special import lpmn, lpmv
import numpy as np
from numpy import sqrt, pi, sin, cos, exp
from math import factorial
import sys

In [2]:
# Use the scipy module
legfunc,  dlegfunc = lpmn(2, 5, 0.5)

In [3]:
def legendre_func(l, m, x):
    """Calculate the Associated Legendre function at x.
    
    Please note that here 'l' and 'm' correspond to 'm' and 'n' in the scipy.special.legendre.
    """
    
    if x > 1:
        print("For x={:.f}, |x| > 1, which is not supported.".format(x))
        sys.exit()
        
    if l < m:
        print("Illegal inpput: l={:d} < m={:d}".format(l, m))
        sys.exit()
    
    if m == l:
        
        if m:
            return (2*m -1) * sqrt(1-x**2) * legendre_func(m-1, m-1, x)
        else:
            return 1
    
    elif l - m == 1:
        return (2 * m + 1) * x * legendre_func(m, m, x)
    
    else:
        return ((2*l-1)*x*legendre_func(l-1, m, x) - (l-1+m)*legendre_func(l-2, m, x)) / (l-m)

In [4]:
print("Comparison of results")
print("Scipy           My")
print(lpmv(2, 5, 0.5), legendre_func(5, 2, 0.5))

Comparison of results
Scipy           My
-4.921874999999999 -4.921875


My calculation of the Legendre function seems correct :-)

In [5]:
def B_coef(l, m, x):
    """
    """
    
    if type(l) is not int:
        
        print("The type of 'l' should be int!")
        sys.exit()
        
    elif type(m) is not int:
        print("The type of 'm' should be int!")
        sys.exit()
    
    if m == 0:
        return 0

    if l == m:
        if m == 1:
            return 1
        else:
            return (2*m - 1) * m / (m - 1) * sqrt(1-x*x) * B_coef(m-1, m-1, x)
        
    elif l - m == 1:
        return (2*m + 1) * x * B_coef(m, m, x)
    
    else:
        return ((2*l-1)*x*B_coef(l-1, m, x) - (l-1+m)*B_coef(l-2, m, x)) / (l-m)

In [6]:
l = 5
m = 2
x = 0.5

print("B:")
print("Calculated:", B_coef(l, m, x))
print("Predicted: ", m * legendre_func(l, m, x) / sqrt(1-x*x))

B:
Calculated: -11.366583424670758
Predicted:  -11.366583424670758


The calculation of $B_{l,m}$ seems good, too.

In [7]:
def A_coef(l, m, x):
    """
    """
    
    if type(l) is not int:
        
        print("The type of 'l' should be int!")
        sys.exit()
        
    elif type(m) is not int:
        print("The type of 'm' should be int!")
        sys.exit()
    
    if m == 0:
        return sqrt(1 - x*x) * B_coef(l, 1, x)
    else:
        return (-x * l * B_coef(l, m, x) + (l+m) * B_coef(l-1, m, x)) / m

In [8]:
print("A:")
print("Calculated:", A_coef(l, m, x))
print("Predicted: ", dlegfunc[-1,-1] * sqrt(1-x*x))

A:
Calculated: 48.30797955485072
Predicted:  48.307979554850725


So does $A_{l,m}$.

In [9]:
def vec_sph_harm_calc(l, m, ra, dec):
    """
    """
    
    fac1 = (2*l+1) / l / (l+1) / (4 * pi) * factorial(l-m) / factorial(l+m)
    fac2 = (-1)**m * sqrt(fac1) * exp(complex(0, m*ra))
    
    x = sin(dec)
    
    facA = fac2 * A_coef(l, m, x)
    facB = fac2 * B_coef(l, m, x)

    t_vec_ra = facA
    t_vec_dc = facB * (0-1j)
    
    s_vec_ra = facB * (0+1j)
    s_vec_dc = facA
    
    
    return t_vec_ra, t_vec_dc, s_vec_ra, s_vec_dc

Then I compare my results with the spherical harmonics wrapper from scipy.special.

In [10]:
from scipy.special import sph_harm

In [11]:
ra = pi / 3
dec = pi / 4

vec_sph_harm_calc(l, m, ra, dec)

((-0.13674626184170208+0.23685147325494535j),
 (0.13398342629807677+0.07735536724014297j),
 (-0.13398342629807677-0.07735536724014297j),
 (-0.13674626184170208+0.23685147325494535j))

In [12]:
fac = (0-1j) * (-1)**m / sqrt(l * (l+1)) * m / sqrt(1-sin(dec)**2)


print("T_{l,m}:", fac * sph_harm(m, l, ra, dec))

T_{l,m}: (0.1339834262980768+0.07735536724014298j)


In [13]:
sph_harm(m, l, ra, dec)

(-0.1497980245304163+0.2594577893601302j)

As it shows, these two results only differ by a factor.